In [ ]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd

from tensorflow.keras.layers import Conv2D, Input, AveragePooling2D, Activation, GlobalAveragePooling2D, Concatenate, BatchNormalization, MaxPooling2D, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers.merge import concatenate
from tensorflow.keras.models import load_model

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from keras import backend as k

import imgaug as ia
from imgaug import augmenters as iaa

num_classes = 200

img_height, img_width = 64, 64

channels = 3

batch_size = 128
epochs = 24
num_train = 100000
num_validation = 10000

## Import Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
!unzip -qq 'tiny-imagenet-200.zip'

--2021-12-20 22:29:07--  http://cs231n.stanford.edu/tiny-imagenet-200.zip
Resolving cs231n.stanford.edu (cs231n.stanford.edu)... 171.64.68.10
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
Saving to: ‘tiny-imagenet-200.zip’

tiny-imagenet-200.z 100%[===================>] 236.61M  18.1MB/s    in 15s     

2021-12-20 22:29:22 (16.3 MB/s) - ‘tiny-imagenet-200.zip’ saved [248100043/248100043]



In [ ]:
val_data = pd.read_csv('./tiny-imagenet-200/val/val_annotations.txt', sep='\t', header=None, 
                       names=['File', 'Class', 'X', 'Y', 'H', 'W'])
val_data.drop(['X', 'Y', 'H', 'W'], axis=1, inplace=True)
val_data.head(5)

,File,Class
0,val_0.JPEG,n03444034
1,val_1.JPEG,n04067472
2,val_2.JPEG,n04070727
3,val_3.JPEG,n02808440
4,val_4.JPEG,n02808440


# Image Augmentation

In [ ]:
def CustomImageDataGen(input_img):

  prob = lambda aug: iaa.Sometimes(0.5, aug)
  
  seq = iaa.Sequential([
      iaa.Fliplr(0.5),      # Horizontal Flip
      iaa.Flipud(0.3),      # Vertical Flip
      prob(iaa.GaussianBlur(sigma=(0, 2.5))),     # Gaussian Blur
      prob(iaa.CropAndPad(
          percent=(-0.1, 0.1),
          pad_mode=ia.ALL,
          pad_cval=(0, 255)
        )),                                       # Crop and Pad
      prob(iaa.Affine(
          scale={"x": (0.9, 1.5), "y": (0.9, 1.5)},     # Scale
          translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},     # Translate
          rotate=(-45, 45),     # Rotate
          shear=(-15, 15),      # Shear
          mode=ia.ALL
      )),      
      prob(iaa.CoarseDropout(
                        (0.05, 0.15), size_percent=(0.03, 0.06),
                        per_channel=0.2
                    )),          # Coarse Dropout
                
      prob(iaa.Multiply((0.8, 1.2), per_channel=0.25)),      # Multiply
      
      prob(iaa.ContrastNormalization((0.75, 1.5), per_channel=0.5)),      # Contrast Normalisation
     ],

     random_order = True) 
  
  output_img = seq.augment_image(input_img)
  return output_img

train_datagen = ImageDataGenerator(rescale=1/255., preprocessing_function = CustomImageDataGen)
valid_datagen = ImageDataGenerator(rescale=1/255.)

In [ ]:
train_generator = train_datagen.flow_from_directory( r'./tiny-imagenet-200/train/', 
                                                    target_size=(img_width, img_height), 
                                                    batch_size=batch_size, 
                                                    class_mode='categorical', 
                                                    shuffle=True, seed=101)

Found 100000 images belonging to 200 classes.


In [ ]:
validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='./tiny-imagenet-200/val/images/', 
                                                         x_col='File', y_col='Class', 
                                                         target_size=(img_width, img_height),
                                                         class_mode='categorical', 
                                                         batch_size=batch_size, 
                                                         shuffle=False, seed=101)

Found 10000 validated image filenames belonging to 200 classes.


# Model

In [ ]:
input = Input(shape=(img_height, img_width, channels))

# Block 1
layer0 = Conv2D(32, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.L2(2e-4))(input)
layer0 = BatchNormalization()(layer0)
layer0 = Activation('relu')(layer0)

skip_connection_1 = layer0

# Block 2
layer1 = Conv2D(128, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.L2(2e-4))(layer0)
layer1 = BatchNormalization()(layer1)
layer1 = Activation('relu')(layer1)

layer2 = Conv2D(128, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.L2(2e-4))(layer1)
layer2 = BatchNormalization()(layer2)
layer2 = Activation('relu')(layer2)

layer3 = Conv2D(128, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.L2(2e-4))(layer2)
layer3 = BatchNormalization()(layer3)
layer3 = Activation('relu')(layer3)

layer4 = Conv2D(128, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.L2(2e-4))(layer3)
layer4 = BatchNormalization()(layer4)
layer4 = Activation('relu')(layer4)

layer5 = concatenate([skip_connection_1, layer4])
layer5 = BatchNormalization()(layer5)
layer5 = Activation('relu')(layer5)
layer5 = MaxPooling2D(pool_size=(2, 2))(layer5)

skip_connection_2 = layer5

# Block 3
layer6 = Conv2D(256, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.L2(2e-4))(layer5)
layer6 = BatchNormalization()(layer6)
layer6 = Activation('relu')(layer6)

layer7 = Conv2D(256, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.L2(2e-4))(layer6)
layer7 = BatchNormalization()(layer7)
layer7 = Activation('relu')(layer7)

layer8 = Conv2D(256, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.L2(2e-4))(layer7)
layer8 = BatchNormalization()(layer8)
layer8 = Activation('relu')(layer8)

layer9 = Conv2D(256, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.L2(2e-4))(layer8)
layer9 = BatchNormalization()(layer9)
layer9 = Activation('relu')(layer9)

layer10 = concatenate([skip_connection_2, layer9])
layer10 = BatchNormalization()(layer10)
layer10 = Activation('relu')(layer10)
layer10 = MaxPooling2D(pool_size=(2, 2))(layer10)

skip_connection_3 = layer10


# Block 4
layer11 = Conv2D(512, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.L2(2e-4))(layer10)
layer11 = BatchNormalization()(layer11)
layer11 = Activation('relu')(layer11)

layer12 = Conv2D(512, (3,3), padding='same', kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.L2(2e-4))(layer11)
layer12 = BatchNormalization()(layer12)
layer12 = Activation('relu')(layer12)

layer13 = Conv2D(512, (3,3), padding='same',kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.L2(2e-4))(layer12)
layer13 = BatchNormalization()(layer13)
layer13 = Activation('relu')(layer13)

layer14 = Conv2D(512, (3,3), padding='same',kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.L2(2e-4))(layer13)
layer14 = BatchNormalization()(layer14)
layer14 = Activation('relu')(layer14)

layer15 = concatenate([skip_connection_3, layer14])
layer15 = BatchNormalization()(layer15)
layer15 = Activation('relu')(layer15)
layer15 = MaxPooling2D(pool_size=(2, 2))(layer15)


layer16 = Conv2D(num_classes, (1,1), padding='same',kernel_initializer="VarianceScaling",kernel_regularizer=tf.keras.regularizers.L2(2e-4))(layer15)
layer16 = GlobalAveragePooling2D()(layer16)

#Output Layer
output = Activation('softmax')(layer16)

In [ ]:
model = tf.keras.models.Model(inputs=[input], outputs=[output])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 conv2d_14 (Conv2D)             (None, 64, 64, 32)   896         ['input_2[0][0]']                
                                                                                                  
 batch_normalization_16 (BatchN  (None, 64, 64, 32)  128         ['conv2d_14[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_17 (Activation)     (None, 64, 64, 32)   0           ['batch_normalization_16[0]

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer= tf.keras.optimizers.RMSprop(learning_rate= 0.0001, epsilon=1e-08),
              metrics=['accuracy'])

In [ ]:
# From https://github.com/bckenstler/CLR
from keras.callbacks import *
from tensorflow.keras.callbacks import *
from tensorflow.keras import backend as K

class CyclicLR(Callback):
    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())

## Train

In [ ]:
clr = CyclicLR(base_lr=0.0001, max_lr=0.0006, step_size=4686., mode='triangular2')
checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath="/content/drive/My Drive/Colab Notebooks/Saved Models_2/model1.hdf5", 
                                                  verbose=1, 
                                                  save_best_only=True, 
                                                  monitor="val_accuracy")

history = model.fit(train_generator,
                    epochs=epochs,
                    steps_per_epoch= num_train // batch_size,
                    validation_steps= num_validation // batch_size,
                    validation_data=validation_generator,
                    callbacks=[clr, checkpointer])
df = pd.DataFrame(history.history)
df.to_csv('train1.csv')
from google.colab import files
files.download('train1.csv')

Epoch 1/24
781/781 [==============================] - ETA: 0s - loss: 5.5115 - accuracy: 0.0500
Epoch 00001: val_accuracy improved from -inf to 0.08524, saving model to /content/drive/My Drive/Colab Notebooks/Saved Models_2/model1.hdf5
781/781 [==============================] - 993s 1s/step - loss: 5.5115 - accuracy: 0.0500 - val_loss: 5.0823 - val_accuracy: 0.0852
Epoch 2/24
781/781 [==============================] - ETA: 0s - loss: 4.8753 - accuracy: 0.1030
Epoch 00002: val_accuracy improved from 0.08524 to 0.13802, saving model to /content/drive/My Drive/Colab Notebooks/Saved Models_2/model1.hdf5
781/781 [==============================] - 993s 1s/step - loss: 4.8753 - accuracy: 0.1030 - val_loss: 4.5750 - val_accuracy: 0.1380
Epoch 3/24
781/781 [==============================] - ETA: 0s - loss: 4.5388 - accuracy: 0.1382
Epoch 00003: val_accuracy improved from 0.13802 to 0.15054, saving model to /content/drive/My Drive/Colab Notebooks/Saved Models_2/model1.hdf5
781/781 [=============

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from keras.models import load_model
new_model = load_model("./drive/My Drive/4040_final_project/Colab Notebooks/Saved Models_2/model1.hdf5")

In [ ]:

clr = CyclicLR(base_lr=0.00001, max_lr=0.00006, step_size=4686., mode='triangular2') #Cyclic learning rate
checkpointer_2 = tf.keras.callbacks.ModelCheckpoint(filepath="/content/drive/My Drive/4040_final_project/Colab Notebooks/Saved Models_2/model2.hdf5", 
                                                    save_best_only=True, 
                                                    monitor="val_accuracy")

history = new_model.fit(train_generator,
                        epochs=epochs,
                        steps_per_epoch= num_train // batch_size,
                        validation_steps= num_validation // batch_size,
                        validation_data=validation_generator,
                        callbacks=[clr, checkpointer_2]
                       )

df = pd.DataFrame(history.history)
df.to_csv('train2.csv')
from google.colab import files
files.download('train2.csv')

Epoch 1/24
781/781 [==============================] - ETA: 0s - loss: 4.1240 - accuracy: 0.5084
Epoch 00001: val_accuracy improved from -inf to 0.57993, saving model to /content/drive/My Drive/4040_final_project/Colab Notebooks/Saved Models_2/model2.hdf5
781/781 [==============================] - 1067s 1s/step - loss: 4.1240 - accuracy: 0.5084 - val_loss: 3.7981 - val_accuracy: 0.5799
Epoch 2/24
781/781 [==============================] - ETA: 0s - loss: 4.0962 - accuracy: 0.5172
Epoch 00002: val_accuracy did not improve from 0.57993
781/781 [==============================] - 1056s 1s/step - loss: 4.0962 - accuracy: 0.5172 - val_loss: 3.8113 - val_accuracy: 0.5772
Epoch 3/24
781/781 [==============================] - ETA: 0s - loss: 4.1005 - accuracy: 0.5179
Epoch 00003: val_accuracy did not improve from 0.57993
781/781 [==============================] - 1049s 1s/step - loss: 4.1005 - accuracy: 0.5179 - val_loss: 3.8194 - val_accuracy: 0.5767
Epoch 4/24
781/781 [========================

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from keras.models import load_model
extended_model = load_model("/content/drive/My Drive/4040_final_project/Colab Notebooks/Saved Models_2/model2.hdf5")

In [ ]:
pred=extended_model.predict(validation_generator)
predicted_class_indices=np.argmax(pred,axis=1)

In [ ]:
import os
class_to_name = dict()
file = open('tiny-imagenet-200/words.txt','r')
data= file.readlines()
for line in data:
  words = line.strip('\n').split('\t')
  class_to_name[words[0]] = words[1].split(',')[0]
file.close()

validation_class_names={}
for _class in validation_generator.class_indices.keys():
  validation_class_names.update({_class : class_to_name[_class]})

from sklearn.metrics import confusion_matrix, classification_report
report = classification_report(validation_generator.classes, 
                               predicted_class_indices,
                               target_names=validation_class_names.values(),
                               digits=4,
                               output_dict=True)
print(report)
df = pd.DataFrame(report).transpose()
df.to_csv('report.csv')
from google.colab import files
files.download('report.csv')

{'goldfish': {'precision': 0.7931034482758621, 'recall': 0.92, 'f1-score': 0.851851851851852, 'support': 50}, 'European fire salamander': {'precision': 0.9130434782608695, 'recall': 0.84, 'f1-score': 0.8749999999999999, 'support': 50}, 'bullfrog': {'precision': 0.5166666666666667, 'recall': 0.62, 'f1-score': 0.5636363636363637, 'support': 50}, 'tailed frog': {'precision': 0.5714285714285714, 'recall': 0.48, 'f1-score': 0.5217391304347826, 'support': 50}, 'American alligator': {'precision': 0.5918367346938775, 'recall': 0.58, 'f1-score': 0.5858585858585857, 'support': 50}, 'boa constrictor': {'precision': 0.4528301886792453, 'recall': 0.48, 'f1-score': 0.46601941747572817, 'support': 50}, 'trilobite': {'precision': 0.7959183673469388, 'recall': 0.78, 'f1-score': 0.7878787878787878, 'support': 50}, 'scorpion': {'precision': 0.6842105263157895, 'recall': 0.52, 'f1-score': 0.5909090909090909, 'support': 50}, 'black widow': {'precision': 0.7916666666666666, 'recall': 0.76, 'f1-score': 0.775

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
clr = CyclicLR(base_lr=0.00001, max_lr=0.00006, step_size=3128., mode='triangular2')
checkpointer_3 = tf.keras.callbacks.ModelCheckpoint(filepath="/content/drive/My Drive/4040_final_project/Colab Notebooks/Saved Models_2/model3.hdf5",
                                                    save_best_only=True, 
                                                    monitor="val_accuracy")

history = extended_model.fit(train_generator,
                                       epochs=epochs,
                                       steps_per_epoch= np.ceil(num_train/batch_size),
                                       validation_steps= np.ceil(num_validation/batch_size),
                                       validation_data=validation_generator,
                                       callbacks=[clr, checkpointer_3]
                                       )
                              
df = pd.DataFrame(history.history)
df.to_csv('train3.csv')
from google.colab import files
files.download('train3.csv')

Epoch 1/24
782/782 [==============================] - 963s 1s/step - loss: 4.0658 - accuracy: 0.5811 - val_loss: 3.9780 - val_accuracy: 0.6063
Epoch 2/24
782/782 [==============================] - 957s 1s/step - loss: 4.0910 - accuracy: 0.5769 - val_loss: 4.0357 - val_accuracy: 0.5995
Epoch 3/24
782/782 [==============================] - 958s 1s/step - loss: 4.1332 - accuracy: 0.5711 - val_loss: 4.1013 - val_accuracy: 0.5838
Epoch 4/24
782/782 [==============================] - 961s 1s/step - loss: 4.1858 - accuracy: 0.5634 - val_loss: 4.1457 - val_accuracy: 0.5811
Epoch 5/24
782/782 [==============================] - 965s 1s/step - loss: 4.2100 - accuracy: 0.5634 - val_loss: 4.1125 - val_accuracy: 0.5933
Epoch 6/24
782/782 [==============================] - 963s 1s/step - loss: 4.1886 - accuracy: 0.5744 - val_loss: 4.1074 - val_accuracy: 0.6012
Epoch 7/24
782/782 [==============================] - 957s 1s/step - loss: 4.1561 - accuracy: 0.5839 - val_loss: 4.0890 - val_accuracy: 0.6058

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
extended_model.save('/content/drive/My Drive/4040_final_project/Colab Notebooks/Saved Models_2/extended_model.hdf5')

In [ ]:
from keras.models import load_model
extended_model = load_model("/content/drive/My Drive/4040_final_project/Colab Notebooks/Saved Models_2/extended_model.hdf5")

In [ ]:
clr = CyclicLR(base_lr=0.000001, max_lr=0.000006, step_size=1564., mode='triangular2') #Cyclic learning rate
checkpointer_4 = tf.keras.callbacks.ModelCheckpoint(filepath="/content/drive/My Drive/4040_final_project/Colab Notebooks/Saved Models_2/model4.hdf5",  
                                                    save_best_only=True, 
                                                    monitor="val_accuracy")

history = extended_model.fit(train_generator,
                             epochs=12,
                             validation_data=validation_generator,
                             callbacks=[clr, checkpointer_4]
                             )
df = pd.DataFrame(history.history)
df.to_csv('train4.csv')
from google.colab import files
files.download('train4.csv')

Epoch 1/12
782/782 [==============================] - 1018s 1s/step - loss: 4.0985 - accuracy: 0.6338 - val_loss: 4.1549 - val_accuracy: 0.6237
Epoch 2/12
782/782 [==============================] - 992s 1s/step - loss: 4.1026 - accuracy: 0.6322 - val_loss: 4.1615 - val_accuracy: 0.6246
Epoch 3/12
782/782 [==============================] - 994s 1s/step - loss: 4.1031 - accuracy: 0.6338 - val_loss: 4.1629 - val_accuracy: 0.6265
Epoch 4/12
782/782 [==============================] - 980s 1s/step - loss: 4.0887 - accuracy: 0.6380 - val_loss: 4.1570 - val_accuracy: 0.6258
Epoch 5/12
782/782 [==============================] - 973s 1s/step - loss: 4.0918 - accuracy: 0.6365 - val_loss: 4.1587 - val_accuracy: 0.6252
Epoch 6/12
782/782 [==============================] - 981s 1s/step - loss: 4.0960 - accuracy: 0.6356 - val_loss: 4.1680 - val_accuracy: 0.6252
Epoch 7/12
782/782 [==============================] - 978s 1s/step - loss: 4.0963 - accuracy: 0.6367 - val_loss: 4.1619 - val_accuracy: 0.625

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>